<!-- # Copyright (c) 2025 takotime808 -->

In [ ]:
# Create a Streamlit app version of the UMAP sampling inference tool
streamlit_script_lines = [
    "import streamlit as st",
    "import numpy as np",
    "import pandas as pd",
    "from scipy.stats import qmc",
    "from sklearn.preprocessing import StandardScaler",
    "from sklearn.cluster import KMeans",
    "from sklearn.metrics import silhouette_score",
    "from scipy.spatial import KDTree",
    "from scipy.stats import entropy",
    "import umap",
    "import plotly.express as px",
    "",
    "st.set_page_config(page_title='UMAP Sampling Inference', layout='wide')",
    "st.title('🔍 UMAP Sampling Method Inference')",
    "st.markdown(\"\"\"This app generates synthetic data from common sampling strategies (Grid, Random, LHS, Sobol),\n",
    "performs UMAP + clustering, and infers which sampling method was used based on structural patterns.\"\"\")",
    "",
    "# Sidebar options",
    "n_samples = st.sidebar.slider('Samples per method (except grid)', 50, 500, 100, step=10)",
    "run_button = st.sidebar.button('Run Inference')",
    "",
    "# Generate sampling data",
    "def generate_data(n=100):",
    "    np.random.seed(42)",
    "    grid = np.array(np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10))).T.reshape(-1, 2)",
    "    lhs = qmc.LatinHypercube(d=2).random(n)",
    "    sobol = qmc.Sobol(d=2).random(n)",
    "    rand = np.random.rand(n, 2)",
    "    X = np.vstack([grid, rand, lhs, sobol])",
    "    labels = ['Grid'] * len(grid) + ['Random'] * n + ['LHS'] * n + ['Sobol'] * n",
    "    df = pd.DataFrame(X, columns=['x1', 'x2'])",
    "    df['TrueSampling'] = labels",
    "    return df",
    "",
    "# Inference function",
    "def classify_sampling(embedding, clusters):",
    "    dists = KDTree(embedding).query(embedding, k=2)[0][:, 1]",
    "    std = np.std(dists)",
    "    sil = silhouette_score(embedding, clusters)",
    "    ent = entropy(np.histogram(dists, bins=30, density=True)[0])",
    "    if std < 0.05 and sil > 0.6:",
    "        return 'Grid', 'Low std and high silhouette -> Grid'",
    "    if std > 0.2 and sil < 0.3:",
    "        return 'Random', 'High std and low silhouette -> Random'",
    "    if 0.05 <= std <= 0.15:",
    "        return ('Sobol', 'Moderate spread, low entropy -> Sobol') if ent < 2.0 else ('LHS', 'Moderate spread, higher entropy -> LHS')",
    "    return 'Uncertain', 'Pattern unclear'",
    "",
    "# Main run",
    "if run_button:",
    "    df = generate_data(n_samples)",
    "    X_scaled = StandardScaler().fit_transform(df[['x1', 'x2']])",
    "    X_umap = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42).fit_transform(X_scaled)",
    "    kmeans = KMeans(n_clusters=4, random_state=42)",
    "    clusters = kmeans.fit_predict(X_umap)",
    "    df_umap = pd.DataFrame(X_umap, columns=['UMAP1', 'UMAP2'])",
    "    df_umap['Cluster'] = clusters",
    "    df_umap['TrueSampling'] = df['TrueSampling']",
    "    method, explanation = classify_sampling(X_umap, clusters)",
    "    st.success(f'**Inferred Sampling Method:** {method}')",
    "    st.caption(f'*Justification:* {explanation}')",
    "    fig = px.scatter(df_umap, x='UMAP1', y='UMAP2', color='Cluster', symbol='TrueSampling',",
    "                    title=f'UMAP 2D Projection - Inferred: {method}', hover_data=['TrueSampling'])",
    "    st.plotly_chart(fig, use_container_width=True)"
]

# Save the Streamlit script
streamlit_path = "umap_sampling_streamlit.py"
with open(streamlit_path, "w") as f:
    f.write("\n".join(streamlit_script_lines))

# streamlit_path.name


In [1]:
# Create a Typer-based CLI script
typer_script_lines = [
    "#!/usr/bin/env python3",
    "# Typer CLI: UMAP Sampling Method Inference",
    "import numpy as np",
    "import pandas as pd",
    "from scipy.stats import qmc",
    "from sklearn.preprocessing import StandardScaler",
    "from sklearn.cluster import KMeans",
    "from sklearn.metrics import silhouette_score",
    "from scipy.spatial import KDTree",
    "from scipy.stats import entropy",
    "import umap",
    "import plotly.express as px",
    "import plotly.io as pio",
    "import typer",
    "",
    "app = typer.Typer()",
    "",
    "def generate_data(n: int = 100):",
    "    np.random.seed(42)",
    "    grid = np.array(np.meshgrid(np.linspace(0, 1, 10), np.linspace(0, 1, 10))).T.reshape(-1, 2)",
    "    lhs = qmc.LatinHypercube(d=2).random(n)",
    "    sobol = qmc.Sobol(d=2).random(n)",
    "    rand = np.random.rand(n, 2)",
    "    X = np.vstack([grid, rand, lhs, sobol])",
    "    labels = ['Grid'] * len(grid) + ['Random'] * n + ['LHS'] * n + ['Sobol'] * n",
    "    df = pd.DataFrame(X, columns=['x1', 'x2'])",
    "    df['TrueSampling'] = labels",
    "    return df",
    "",
    "def classify_sampling(embedding, clusters):",
    "    dists = KDTree(embedding).query(embedding, k=2)[0][:, 1]",
    "    std = np.std(dists)",
    "    sil = silhouette_score(embedding, clusters)",
    "    ent = entropy(np.histogram(dists, bins=30, density=True)[0])",
    "    if std < 0.05 and sil > 0.6:",
    "        return 'Grid', 'Low std and high silhouette -> Grid'",
    "    if std > 0.2 and sil < 0.3:",
    "        return 'Random', 'High std and low silhouette -> Random'",
    "    if 0.05 <= std <= 0.15:",
    "        return ('Sobol', 'Moderate spread, low entropy -> Sobol') if ent < 2.0 else ('LHS', 'Moderate spread, higher entropy -> LHS')",
    "    return 'Uncertain', 'Pattern unclear'",
    "",
    "@app.command()",
    "def main(",
    "    html: str = typer.Option(\"umap_output.html\", help=\"Output HTML file path\"),",
    "    png: str = typer.Option(\"umap_output.png\", help=\"Output PNG file path\"),",
    "    n: int = typer.Option(100, help=\"Number of samples for each method (except grid)\")",
    "):",
    "    df = generate_data(n)",
    "    X_scaled = StandardScaler().fit_transform(df[['x1', 'x2']])",
    "    X_umap = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42).fit_transform(X_scaled)",
    "    kmeans = KMeans(n_clusters=4, random_state=42)",
    "    clusters = kmeans.fit_predict(X_umap)",
    "    df_umap = pd.DataFrame(X_umap, columns=['UMAP1', 'UMAP2'])",
    "    df_umap['Cluster'] = clusters",
    "    df_umap['TrueSampling'] = df['TrueSampling']",
    "    method, explanation = classify_sampling(X_umap, clusters)",
    "    print(f'Inferred Sampling Method: {method}')",
    "    print(f'Justification: {explanation}')",
    "    fig = px.scatter(df_umap, x='UMAP1', y='UMAP2', color='Cluster', symbol='TrueSampling',",
    "                     title=f'UMAP 2D Projection - Inferred: {method}', hover_data=['TrueSampling'])",
    "    fig.write_html(html)",
    "    pio.write_image(fig, png, width=1000, height=800)",
    "    fig.show()",
    "",
    "if __name__ == \"__main__\":",
    "    app()"
]

# Save the Typer CLI script
# typer_script_path = Path(tempfile.gettempdir()) / "umap_sampling_typer_cli.py"
typer_script_path = "umap_sampling_typer_cli.py"
with open(typer_script_path, "w") as f:
    f.write("\n".join(typer_script_lines))

# typer_script_path.name
